---
title: Neural Networks (Draft)
---

So far, we have explored basics of PyTorch and used linear regression example to showcase functionality within PyTorch. In this chapter, we will start building on that knowledge and use PyTorch to build neural networks. We will be using `MNIST png` dataset from [Kaggle](https://www.kaggle.com/datasets/jidhumohan/mnist-png). There are also CSV version of MNIST, but just to give more real experience we are using `png` format.

## Downloading Data from Kaggle 

Here are few steps you need to perform before we download the data - 

- If you don’t have a Kaggle account, you can make one for free [here](https://www.kaggle.com/).
- To download the dataset, you will need `kaggle` installed, you can run the following command in notebook or CLI.
    ```{python}
    !pip install kaggle >> /dev/null
    ```
- Have a `kaggle.json` stored in `~/.kaggle`. You can get your token by going to Your Profile -> Account -> Create New API Token.


Once you have the above three steps done, run the API command provided:

In [14]:
!kaggle datasets download -d jidhumohan/mnist-png -p "../data/"

 87%|█████████████████████████████████     | 51.0M/58.6M [00:01<00:00, 33.4MB/s]
100%|██████████████████████████████████████| 58.6M/58.6M [00:01<00:00, 33.9MB/s]


To inspect file system we will use [fastcore](https://fastcore.fast.ai/) `Path` function, as it extends python `Path` functionality and makes it easier to inspect directories and folder.

In [2]:
from fastcore.xtras import Path
zip_path = Path("../data/mnist-png.zip")
zip_path.exists() # Check if the file exist

True

Data is now saved to the `mnist-png.zip` file here on our local system in `../data` folder. Now we will use `zipfile` package to unzip the data. 

:::{.callout-warning}
This following code will run for some time as its unzipping 60,000 PNG images.
:::

In [11]:
# Output directory
dPath = Path("../data/")

# Unzipping data file in output directory
import zipfile
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(str(dPath))


# Removing the original zip file
zip_path.unlink()

# Removing duplicate folder in the unzipped data
import shutil
dPath = dPath/'mnist_png'
shutil.rmtree(dPath/'mnist_png')

In [19]:
dPath.ls()

(#2) [Path('../data/mnist_png/testing'),Path('../data/mnist_png/training')]

Data contains of two folder `trainingSet` and `testSet`.

In [20]:
(dPath/'training').ls()

(#10) [Path('../data/mnist_png/training/0'),Path('../data/mnist_png/training/1'),Path('../data/mnist_png/training/2'),Path('../data/mnist_png/training/3'),Path('../data/mnist_png/training/4'),Path('../data/mnist_png/training/5'),Path('../data/mnist_png/training/6'),Path('../data/mnist_png/training/7'),Path('../data/mnist_png/training/8'),Path('../data/mnist_png/training/9')]

Within training folder there is a folder for each digit from `0` to `9`

Within each there are images.

In [21]:
(dPath/'training/0').ls()

(#5923) [Path('../data/mnist_png/training/0/1.png'),Path('../data/mnist_png/training/0/1000.png'),Path('../data/mnist_png/training/0/10005.png'),Path('../data/mnist_png/training/0/10010.png'),Path('../data/mnist_png/training/0/10022.png'),Path('../data/mnist_png/training/0/10025.png'),Path('../data/mnist_png/training/0/10026.png'),Path('../data/mnist_png/training/0/10045.png'),Path('../data/mnist_png/training/0/10069.png'),Path('../data/mnist_png/training/0/10071.png')...]

Let's load few image.

In [22]:
from PIL import Image
from IPython.display import display
for img in [Image.open((dPath/'training/0').ls()[0]), 
            Image.open((dPath/'training/1').ls()[0]),
            Image.open((dPath/'training/2').ls()[0]),
            Image.open((dPath/'training/3').ls()[0])]: 
    display(img)

## Creating Dataset Object using Torchvision

In [23]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from tqdm.auto import tqdm

In [24]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(), 
    transforms.Lambda(lambda x: torch.flatten(x))
    ])

## Create a dataset
train_ds = datasets.ImageFolder(root = dPath/'training/', 
                                      transform=transform)

test_ds = datasets.ImageFolder(root=dPath/'testing', transform=transform)

print(f'Length of train dataset: {len(train_ds)}, test_dataset: {len(test_ds)}')
print(f'One object: Image Tensor of shape {train_ds[0][0].shape}, Label: {train_ds[0][1]}')

Length of train dataset: 60000, test_dataset: 10000
One object: Image Tensor of shape torch.Size([784]), Label: 0


Create a dataloader using `torch.utils.data.DataLoader` function.

In [26]:
import os
train_dls = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=int(os.cpu_count()/2))
test_dls = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=int(os.cpu_count()/2))

Let's look at one batch

In [27]:
batch = next(iter(train_dls))
batch[0].shape, batch[1]

(torch.Size([128, 784]),
 tensor([9, 8, 0, 2, 1, 5, 4, 0, 0, 0, 1, 7, 7, 4, 4, 4, 0, 8, 1, 4, 4, 3, 9, 0,
         4, 6, 9, 3, 7, 6, 4, 8, 6, 4, 1, 9, 7, 5, 9, 3, 6, 0, 8, 2, 7, 5, 3, 3,
         8, 7, 1, 3, 5, 7, 6, 4, 7, 2, 6, 3, 4, 7, 7, 5, 5, 6, 2, 4, 3, 5, 3, 1,
         2, 6, 3, 9, 6, 6, 9, 9, 3, 2, 5, 3, 5, 4, 0, 8, 0, 8, 3, 9, 4, 7, 2, 3,
         0, 8, 0, 9, 7, 7, 1, 2, 2, 4, 6, 3, 4, 7, 0, 7, 6, 1, 2, 3, 5, 4, 6, 9,
         1, 5, 9, 6, 7, 3, 2, 7]))

In [28]:
class MLP(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_in, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, n_out)
        )
    def forward(self, x):
        return self.model(x)

In [67]:
## Training loop
def train_one_epoch(model, data_loader, optimizer, loss_func):
    total_loss, nums = 0, 0
    for batch in tqdm(iter(data_loader)):
        ## Taking one mini-batch
        xb, yb = batch[0].to(dev), batch[1].to(dev)
        y_pred = model.forward(xb)
        
        ## Calculation mean square error per min-batch
        loss = loss_func(y_pred, yb)
        total_loss += loss.item()
        nums += len(yb)
        ## Computing gradients per mini-batch
        loss.backward()
        
        ## Update model parameters and zero grad
        optimizer.step()
        optimizer.zero_grad()
    
    return  total_loss/nums
        
def validate_one_epoch(model, data_loader, loss_func):
    loss, nums, acc = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(iter(data_loader)):
            xb, yb = batch[0].to(dev), batch[1].to(dev)
            y_pred = model.forward(xb)
            loss += loss_func(y_pred, yb).item()
            acc += sum(y_pred.argmax(axis=1) == yb).item()
            nums += len(yb)
    return loss/nums, acc/nums

In [73]:
model = MLP(784,10)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_func = nn.CrossEntropyLoss()
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(dev)

Let's test our untrained model accuracy.

In [78]:
test_loss, test_acc = validate_one_epoch(model=model, data_loader=test_dls, loss_func=loss_func)
print(f"Random model: Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

  0%|          | 0/79 [00:00<?, ?it/s]

Random model: Test Loss: 0.0182, Test Accuracy: 0.1208


Let's train the model now for 5 epochs and we will use `AdamW` optimizer instead of `SGD` from `torch.optim` module.

In [72]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
for epoch in range(5):    
    # run one training loop
    train_loss = train_one_epoch(model=model, data_loader=train_dls, optimizer=optimizer, loss_func=loss_func)
    test_loss, test_acc = validate_one_epoch(model=model, data_loader=test_dls, loss_func=loss_func)
    print(f"Epoch {epoch},Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 0,Train Loss: 0.0028, Test Loss: 0.0014, Test Accuracy: 0.9511


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 3,Train Loss: 0.0005, Test Loss: 0.0006, Test Accuracy: 0.9744


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch 4,Train Loss: 0.0004, Test Loss: 0.0006, Test Accuracy: 0.9762


We can see after 5 epochs we have a fairly accurate model with 97.62% accuracy as compared to random prediction of 12%.